# Provision

## Initialize
Let's start the Automation Provision

In [1]:
from nornir import InitNornir
from nornir.core import configuration
from nornir.plugins.runners import ThreadedRunner, SerialRunner
from nornir.core.plugins.inventory import InventoryPluginRegister
from nornir.core.filter import F
import logging

from nornir_utils.plugins.functions import print_result
from nornir_utils.plugins.tasks.files import write_file
from nornir_napalm.plugins.tasks import *
from nornir_netmiko.tasks import *
from nornir_jinja2.plugins.tasks import *
from mytasks import *

## Create the Nornir Object

In [2]:
%%time
InventoryPluginRegister.register("ThreadedRunner", ThreadedRunner)
nr = InitNornir(
    config_file='config.yaml',
    dry_run=False)
nr = nr.filter(F(groups__contains="IOS"))


CPU times: user 129 ms, sys: 7.8 ms, total: 137 ms
Wall time: 141 ms


---
## Start the Tasks

### check connections

In [3]:
%%time
result = nr.run(
    task=connection_check,
    on_good=True,
    on_failed=True,
    )
print_result(result, severity_level=logging.WARNING)

if result.failed:
    print(result.failed_hosts)

connection_check****************************************************************
* ACCESS1 ** changed : False ***************************************************
^^^^ END connection_check ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* ACCESS2 ** changed : False ***************************************************
^^^^ END connection_check ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* BR1 ** changed : False *******************************************************
^^^^ END connection_check ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* BR2 ** changed : False *******************************************************
^^^^ END connection_check ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* CORE1 ** changed : False *****************************************************
^^^^ END connection_check ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* CORE2 ** changed : False *****************************************************
^^^^ END connection_check ^^

## Backup Current Configuration


In [4]:
%%time
result = nr.run(
    task=save_show,
    command='show running',
    filename_mark='changed',
    on_good=True,
    on_failed=True,
    )
print_result(result, severity_level=logging.WARNING)

save_show***********************************************************************
* ACCESS1 ** changed : True ****************************************************
^^^^ END save_show ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* ACCESS2 ** changed : True ****************************************************
^^^^ END save_show ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* BR1 ** changed : True ********************************************************
^^^^ END save_show ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* BR2 ** changed : True ********************************************************
^^^^ END save_show ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* CORE1 ** changed : True ******************************************************
^^^^ END save_show ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* CORE2 ** changed : True ******************************************************
^^^^ END save_show ^^^^^^^^^

---
## Configure Links 

In [5]:
result = nr.run(
    task=jinjia2_write_deploy,
    template_n='interface',
    on_good=True,
    on_failed=True,
    )
print_result(result, severity_level=logging.INFO)

jinjia2_write_deploy************************************************************
* ACCESS1 ** changed : True ****************************************************
vvvv jinjia2_write_deploy ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
vvvv render_configs ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
---- template_file ** changed : False ------------------------------------------ INFO
interface loopback0
 description Router-id
 no shut
 ip address 2.1.3.1 255.255.255.255
interface e0/1
 description To CORE1
 no shut
 ip address 2.23.11.3 255.255.255.0
interface e0/2
 description To HOST
 no shut
 ip address 2.34.101.3 255.255.255.0
interface e0/3
 description To CORE2
 no shut
 ip address 2.23.21.3 255.255.255.0

^^^^ END render_configs ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv write_configs ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
---- write_file ** changed : False -------------------------------------------

---
## Ping Test

In [6]:
result = nr.run(
    task=napalm_ping,
    dest='10.0.137.1',
    vrf='mgmt'
    )
print_result(result, severity_level=logging.INFO)

napalm_ping*********************************************************************
* ACCESS1 ** changed : False ***************************************************
vvvv napalm_ping ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'success': { 'packet_loss': 0,
               'probes_sent': 5,
               'results': [ {'ip_address': '10.0.137.1', 'rtt': 0.0},
                            {'ip_address': '10.0.137.1', 'rtt': 0.0},
                            {'ip_address': '10.0.137.1', 'rtt': 0.0},
                            {'ip_address': '10.0.137.1', 'rtt': 0.0},
                            {'ip_address': '10.0.137.1', 'rtt': 0.0}],
               'rtt_avg': 1.0,
               'rtt_max': 1.0,
               'rtt_min': 1.0,
               'rtt_stddev': 0.0}}
^^^^ END napalm_ping ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* ACCESS2 ** changed : False ***************************************************
vvvv napalm_ping ** changed : False vvvvvvvvv

---
## Save Config


In [8]:
result = nr.run(
    task=netmiko_save_config,
    cmd="write"
    )
print_result(result, severity_level=logging.INFO)

netmiko_save_config*************************************************************
* ACCESS1 ** changed : True ****************************************************
vvvv netmiko_save_config ** changed : True vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
write

Building configuration...
[OK]
ACCESS1#
^^^^ END netmiko_save_config ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* ACCESS2 ** changed : True ****************************************************
vvvv netmiko_save_config ** changed : True vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
write
Building configuration...
[OK]
ACCESS2#
^^^^ END netmiko_save_config ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* BR1 ** changed : True ********************************************************
vvvv netmiko_save_config ** changed : True vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
write

Building configuration...
[OK]
BR1#
^^^^ END netmiko_save_config ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
* BR2 ** changed : True ***************

---
## RollBack

In [10]:
def config_rollback(task, folder_name: str):
    task.run(
        task=netmiko_file_transfer,
        source_file=f"{folder_name}/{task.host.name}.cfg",
        dest_file='running-config',
        overwrite_file=True,
    )

result = nr.run(
    task=config_rollback,
    folder_name = "output/show running-init"   ## input folder name
)
print_result(result, severity_level=logging.INFO)

config_rollback*****************************************************************
